In [1]:
import sys
sys.path.append("../Tools/")

from vnstock import *

from function import *
from operators import *

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests


In [2]:
ticket = ['ACB', 'BID', 'BVH', 'CTG', 'FPT', 'GAS', 'GVR', 'HDB', 'HPG', 'MBB', 'MSN', 'MWG', 'NVL', 'PDR', 'PLX', 'POW', 'SAB', 'SSI', 'STB', 'TCB', 'TPB', 'VCB', 'VHM', 'VIB', 'VIC', 'VJC', 'VNM', 'VPB', 'VRE']

### Field

In [30]:
data = pd.DataFrame(index=range(120))
for i in ticket:
    try:
        a = ticker_overview(i)["industry"][0]
        if a not in data.columns:
            data[a]=financial_report(symbol=i, report_type='CashFlow', frequency='Quarterly')["CHỈ TIÊU"]
    except Exception:
        continue
di = data.isna().sum(axis=1)>=9
di=di[di==True].index
data.drop(di,axis=0,inplace=True)
data.to_excel("field/cf_field.xlsx")    


### IncomeStatement

In [5]:
data = pd.DataFrame()
for i in ticket:
    try:
        df = financial_report(symbol=i, report_type='IncomeStatement', frequency='Quarterly')
        df["ticker"] = i 
        df.index = [ticker_overview(i)["industry"][0]]*len(df.index)
        data = pd.concat([data, df])
    except Exception:
        continue
df = data.groupby([data.index,"CHỈ TIÊU","ticker"]).agg(sum)

In [10]:
df = pd.read_excel("income.xlsx").ffill()
df=df.set_index(df.columns[0])
df = df.groupby([df.index,"CHỈ TIÊU","ticker"]).agg(sum).T["Ngân hàng"]

close = pd.read_csv("../Data/close.csv", index_col="TradingDate", parse_dates=True)
df1 = pd.DataFrame(np.nan, index=close.index, columns=close.columns)
df1.index = df1.index.to_period('Q').strftime('Q%q %Y')
df=df["Chi phí dự phòng rủi ro tín dụng"]
df1.loc[df.index,df.columns]=df.loc[df.index,df.columns]

df1.ffill(inplace=True)
df1.fillna(0,inplace=True)

### Balance sheet

In [48]:
data = pd.DataFrame()
for i in ticket:
    try:
        df = financial_report(symbol=i, report_type='BalanceSheet', frequency='Quarterly')
        df["ticker"] = i 
        df.index = [ticker_overview(i)["industry"][0]]*len(df.index)
        data = pd.concat([data, df])
    except Exception:
        continue
df = data.groupby([data.index,"CHỈ TIÊU","ticker"]).agg(sum)    

### CashFlow

In [51]:
data = pd.DataFrame()
for i in ticket:
    try:
        df = financial_report(symbol=i, report_type='CashFlow', frequency='Quarterly')
        df["ticker"] = i 
        df.index = [ticker_overview(i)["industry"][0]]*len(df.index)
        data = pd.concat([data, df])
    except Exception:
        continue
df = data.groupby([data.index,"CHỈ TIÊU","ticker"]).agg(sum)    